In [1]:
import hid
import struct
import time
import pyautogui

# --- PyAutoGUI 优化设置 ---
pyautogui.MINIMUM_DURATION = 0
pyautogui.MINIMUM_SLEEP = 0
pyautogui.PAUSE = 0
pyautogui.FAILSAFE = False

class XboxController:
    # --- [最终修正] 根据你的精确报文数据得出的按钮映射 ---
    # 你的控制器将按钮和DPAD放在了同一个字节 [4]
    # bit 0-3 是 DPAD, bit 4-7 是 A,B,X,Y
    BUTTON_MAP = {
        4: "A",
        5: "B",
        6: "X",
        7: "Y",
    }
    # 肩键等在另一个字节 [5]
    BUTTON_MAP_2 = {
        0: "DPAD_UP",
        1: "DPAD_DOWN",
        2: "DPAD_LEFT",
        3: "DPAD_RIGHT",
        4: "LB",
        5: "RB",
        6: "LS",
        7: "RS",
    }

    def __init__(self, vendor_id=0x045E, product_id=0x0B12):
        self.device = None
        try:
            self.device = hid.device()
            self.device.open(vendor_id, product_id)
            self.device.set_nonblocking(True)
            print("Connected:", self.device.get_manufacturer_string(),
                  self.device.get_product_string())
        except OSError as e:
            print(f"Error opening device: {e}")
            self.device = None

    def close(self):
        if self.device:
            self.device.close()

    def read(self):
        if not self.device:
            return None
            
        data = self.device.read(64)
        if not data or len(data) < 18:
            return None

        raw = bytes(data)

        # ----- [关键] 根据你的报文进行的精确解析 -----
        
        # 按钮 (DPAD, A, B, X, Y) 在字节 [4]
        buttons1_raw = raw[4]
        
        # 其他按钮 (LB, RB, Menu, View, etc.) 在字节 [5]
        buttons2_raw = raw[5]

        # 扳机键 LT/RT (10-bit unsigned) 在字节 [6]-[9]
        lt, rt = struct.unpack_from("<HH", raw, 6)

        # 摇杆 (16-bit signed) 在字节 [10]-[17]
        lx, ly, rx, ry = struct.unpack_from("<hhhh", raw, 10)
        
        # Y 轴向上为正，符合直觉，但 pyautogui 向下为正，所以我们在主程序中处理
        # 你的报文中，向上推摇杆已经是正值，所以这里不需要反转
        # ly, ry = -ly, -ry 

        # 合并所有按钮状态
        buttons = self._decode_buttons(buttons1_raw, self.BUTTON_MAP)
        buttons.update(self._decode_buttons(buttons2_raw, self.BUTTON_MAP_2))

        return {
            "buttons": buttons,
            "lt_norm": lt / 1023.0,
            "rt_norm": rt / 1023.0,
            "lx_norm": self._normalize_axis(lx),
            "ly_norm": self._normalize_axis(ly),
            "rx_norm": self._normalize_axis(rx),
            "ry_norm": self._normalize_axis(ry),
        }

    def _decode_buttons(self, bitmask, button_map):
        return {name: bool(bitmask & (1 << bit)) for bit, name in button_map.items()}

    def _normalize_axis(self, v):
        v = int(v)
        if v < 0: return max(-1.0, v / 32768.0)
        else: return min(1.0, v / 32767.0)


if __name__ == "__main__":
    # --- 新增：滚动速度控制 ---
    # 这个值代表每次循环滚动多少“单位”，可以根据需要调整
    SCROLL_SPEED = 5
    
    MOUSE_SENSITIVITY = 25 
    DEADZONE = 0.15

    try:
        xbox = XboxController() 
        if not xbox.device:
            raise OSError("Controller not found or could not be opened.")

        print("\nController decoded successfully! Mouse control is active.")
        # --- 修改：更新提示信息 ---
        print("A = Left Click | B = Right Click | LB/RB = Scroll | Ctrl+C to exit.")
        print("-" * 50)

        a_button_pressed = False
        b_button_pressed = False
        
        while True:
            state = xbox.read()
            if state:
                # --- 鼠标移动 ---
                lx = state['lx_norm']
                ly = state['ly_norm']
                
                if abs(lx) < DEADZONE: lx = 0
                if abs(ly) < DEADZONE: ly = 0
                
                if lx != 0 or ly != 0:
                    x_move = (lx ** 3) * MOUSE_SENSITIVITY
                    # 你的报文向上推摇杆是正值，而pyautogui向下是正值，所以需要一个负号
                    y_move = -(ly ** 3) * MOUSE_SENSITIVITY
                    pyautogui.move(x_move, y_move)
                
                # --- 鼠标点击 ---
                if state['buttons'].get('A') and not a_button_pressed:
                    pyautogui.mouseDown(button='left')
                    a_button_pressed = True
                elif not state['buttons'].get('A') and a_button_pressed:
                    pyautogui.mouseUp(button='left')
                    a_button_pressed = False
                    
                if state['buttons'].get('B') and not b_button_pressed:
                    pyautogui.mouseDown(button='right')
                    b_button_pressed = True
                elif not state['buttons'].get('B') and b_button_pressed:
                    pyautogui.mouseUp(button='right')
                    b_button_pressed = False

                # --- 新增：鼠标滚动功能 ---
                # 按住 RB (Right Bumper) 向上滚动
                if state['buttons'].get('RB'):
                    pyautogui.scroll(SCROLL_SPEED)
                
                # 按住 LB (Left Bumper) 向下滚动
                if state['buttons'].get('LB'):
                    pyautogui.scroll(-SCROLL_SPEED)
                
                # --- 调试输出 ---
                pressed_buttons = sorted([name for name, pressed in state["buttons"].items() if pressed])
                print(
                    f"Stick:({state['lx_norm']:.2f}, {state['ly_norm']:.2f}) "
                    f"LT:{state['lt_norm']:.2f} RT:{state['rt_norm']:.2f} "
                    f"Buttons: {pressed_buttons}      ", end='\r'
                )

            time.sleep(0.001) # 可以尝试更小的延迟

    except OSError as e:
        print(f"\nError: {e}")
    except KeyboardInterrupt:
        print("\nExiting.")
    finally:
        if 'xbox' in locals() and xbox:
            xbox.close()

Connected: Microsoft Controller

Controller decoded successfully! Mouse control is active.
A = Left Click | B = Right Click | LB/RB = Scroll | Ctrl+C to exit.
--------------------------------------------------
Stick:(0.03, -0.01) LT:0.00 RT:0.00 Buttons: []          
Exiting.
